# 1. Reading and Analysing

In [11]:
import pyodbc
import pandas as pd

server = 'DELL\\SQLEXPRESS'
database = 'NEHADB'

connection_string = f"DRIVER={{ODBC Driver 17 for SQL Server}};SERVER={server};DATABASE={database};Trusted_Connection=yes;"
conn = pyodbc.connect(connection_string)

query = "SELECT * FROM dbo.CS_FUND"
cursor = conn.cursor()
cursor.execute(query)

rows = cursor.fetchall()
columns = [column[0] for column in cursor.description]

df = pd.DataFrame.from_records(rows, columns=columns)

print(df.head())

print("Data Overview:\n", df.head())
print("\nStatistics:\n", df.describe(include='all'))


  ACCT_CD                                          ACCT_NAME ACCT_SHT_NAME  \
0       6      FCA Canada Inc. Elected Master Trust (Equity)         k_006   
1    006x  FCA Canada Inc. Elected Master Trust (Equity)(...          NULL   
2       7       Canadian Pacific Airline Pilots' Association          NULL   
3    007x  Canadian Pacific Airline Pilots' Association (...          NULL   
4    008x                Grosvenor International Canada Ltd.          NULL   

  ACCT_TYP_CD PARENT_CHILD_FLAG CRRNCY_CD STATE_CD CNTRY_CD  MKT_VAL  \
0           F              NULL       CAD     NULL     NULL      0.0   
1           F              NULL       CAD     NULL     NULL      NaN   
2           F              NULL       CAD     NULL     NULL      0.0   
3           F              NULL       CAD     NULL     NULL      NaN   
4           F              NULL       CAD     NULL     NULL      NaN   

   NET_ASSETS  ...  TARGET_DURATION FACTOR_MODEL_HIERARCHY_CD  \
0         0.0  ...             NU

# 2. Datatype Suggestion

In [2]:
dtype_mapping = {
    'int64': 'INT',
    'float64': 'FLOAT',
    'object': 'VARCHAR(MAX)',  # String type
    'datetime64[ns]': 'DATETIME',
    'bool': 'BIT'
}

suggested_dtypes = {col: dtype_mapping[str(df[col].dtype)] for col in df.columns}

print("Suggested Data Types for SQL Server:")
print(suggested_dtypes)


Suggested Data Types for SQL Server:
{'ACCT_CD': 'VARCHAR(MAX)', 'ACCT_NAME': 'VARCHAR(MAX)', 'ACCT_SHT_NAME': 'VARCHAR(MAX)', 'ACCT_TYP_CD': 'VARCHAR(MAX)', 'PARENT_CHILD_FLAG': 'VARCHAR(MAX)', 'CRRNCY_CD': 'VARCHAR(MAX)', 'STATE_CD': 'VARCHAR(MAX)', 'CNTRY_CD': 'VARCHAR(MAX)', 'MKT_VAL': 'FLOAT', 'NET_ASSETS': 'FLOAT', 'TOT_ASSETS': 'FLOAT', 'FUND_SHRS_OUTST': 'VARCHAR(MAX)', 'TOT_COST': 'FLOAT', 'NET_CASH': 'VARCHAR(MAX)', 'TOT_INVSTMNTS': 'FLOAT', 'NET_FUNDS_AVAIL': 'VARCHAR(MAX)', 'LIABILITIES': 'VARCHAR(MAX)', 'AMRTZD_COST': 'VARCHAR(MAX)', 'AVG_COST': 'VARCHAR(MAX)', 'OTH_ASSET': 'VARCHAR(MAX)', 'DIV_RECEIVED': 'VARCHAR(MAX)', 'INT_RECEIVED': 'VARCHAR(MAX)', 'CNTRBS': 'VARCHAR(MAX)', 'PMNTS': 'VARCHAR(MAX)', 'TRANSFERS': 'VARCHAR(MAX)', 'CASH_BAL_SOD': 'VARCHAR(MAX)', 'RECVB_SEC_SOLD': 'VARCHAR(MAX)', 'PAYBL_SEC_PURCH': 'VARCHAR(MAX)', 'RECVB_FUND_SHRS_SOLD': 'VARCHAR(MAX)', 'PAYBL_FUND_SHRS_LIQD': 'VARCHAR(MAX)', 'DIV_RCVBLE': 'VARCHAR(MAX)', 'ACCRUED_INT_INCM': 'VARCHAR(MAX)',

# 3. Dropping NULL Columns

In [3]:
import pandas as pd

columns_to_keep = []

for column in df.columns:
    non_null_values = df[column].dropna().tolist()
    if non_null_values:  # Check if the list is not empty
        columns_to_keep.append(column)
        print(f"{column}: {non_null_values}")
        
df_cleaned = df[columns_to_keep]


IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [5]:
import pandas as pd
import numpy as np

df = df.replace("", np.nan)
df_cleaned = df.dropna(axis=1, how='all')
print("Columns kept:", df_cleaned.columns.tolist())


Columns kept: ['ACCT_CD', 'ACCT_NAME', 'ACCT_SHT_NAME', 'ACCT_TYP_CD', 'PARENT_CHILD_FLAG', 'CRRNCY_CD', 'STATE_CD', 'CNTRY_CD', 'MKT_VAL', 'NET_ASSETS', 'TOT_ASSETS', 'FUND_SHRS_OUTST', 'TOT_COST', 'NET_CASH', 'TOT_INVSTMNTS', 'NET_FUNDS_AVAIL', 'LIABILITIES', 'AMRTZD_COST', 'AVG_COST', 'OTH_ASSET', 'DIV_RECEIVED', 'INT_RECEIVED', 'CNTRBS', 'PMNTS', 'TRANSFERS', 'CASH_BAL_SOD', 'RECVB_SEC_SOLD', 'PAYBL_SEC_PURCH', 'RECVB_FUND_SHRS_SOLD', 'PAYBL_FUND_SHRS_LIQD', 'DIV_RCVBLE', 'ACCRUED_INT_INCM', 'ACCRUED_EXP_INCM', 'DIV_PAYBL', 'OTH_LIAB', 'EQTY_ACCT', 'SYST_OF_REFERENCE', 'NEXPERT_INST', 'THEME_1', 'THEME_2', 'THEME_3', 'LOT_SIZE', 'ALLOW_BUY', 'ALLOW_SELL', 'ALLOW_NEG_CASH', 'MANAGER', 'MODEL_PRIV', 'DEFAULT_MODEL_CD', 'START_BATCH_VIOL', 'SHAW_MANAGER_CD', 'SHAW_CLASS_CD', 'CRD_LOCK', 'ERISA_ELIGIBLE', 'INDEX_SEC_ID', 'COMPLIANCE_TOLERANCE', 'APPLY_GUIDELINES', 'GAIN_LOSS_SENSITIVE', 'SHORT_SHORT_GAINS', 'GROSS_INCOME', 'TAX_LOT_SELL_CNVTN', 'BANK_ACCT_NUM', 'CUSTODIAN_CD', 'DEFAULT

# 4. Finding Duplicates and dropping them

In [6]:

if 'ACCT_CD' in df.columns:
    print("\nAnalyzing 'ACCT_CD' column for duplicates...")
    
    duplicate_count = df['ACCT_CD'].duplicated().sum()
    print(f"Number of duplicate entries in 'ACCT_CD': {duplicate_count}")
    
    if duplicate_count > 0:
        df_cleaned = df.drop_duplicates(subset='ACCT_CD', keep='first').reset_index(drop=True)
        print("\nDuplicates removed. Altered DataFrame:")
        print(df_cleaned.head())
    else:
        print("\nNo duplicates found in 'ACCT_CD'.")
else:
    print("\n'ACCT_CD' column not found in the DataFrame.")


Analyzing 'ACCT_CD' column for duplicates...
Number of duplicate entries in 'ACCT_CD': 0

No duplicates found in 'ACCT_CD'.


In [7]:
df_cleaned

,ACCT_CD,ACCT_NAME,ACCT_SHT_NAME,ACCT_TYP_CD,PARENT_CHILD_FLAG,CRRNCY_CD,STATE_CD,CNTRY_CD,MKT_VAL,NET_ASSETS,...,TARGET_DURATION,FACTOR_MODEL_HIERARCHY_CD,BLOCK_MONITOR_TYPE_CD,LAST_MDL_CHANGE_PROCESS_DATE,TARGET_CASH_PCT,DEF_STRATEGY_CHAIN_ID,DEF_BENCH_TYPE3,DEF_BENCH_CD3,DEF_BENCH_ID3,STRATEGY_LOT_ENABLED_IND
0,6,FCA Canada Inc. Elected Master Trust (Equity),k_006,F,NULL,CAD,NULL,NULL,0.0,0.0000,...,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,N
1,006x,FCA Canada Inc. Elected Master Trust (Equity)(...,NULL,F,NULL,CAD,NULL,NULL,NaN,0.0000,...,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,N
2,7,Canadian Pacific Airline Pilots' Association,NULL,F,NULL,CAD,NULL,NULL,0.0,0.0000,...,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,N
3,007x,Canadian Pacific Airline Pilots' Association (...,NULL,F,NULL,CAD,NULL,NULL,NaN,0.0000,...,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,N
4,008x,Grosvenor International Canada Ltd.,NULL,F,NULL,CAD,NULL,NULL,NaN,0.0000,...,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,N
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20588,x0145x,Gestion Marc Valois Inc. USD (Closed),NULL,F,NULL,USD,NULL,NULL,NaN,0.0000,...,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,N
20589,x014x,Gestion Marc Valois Inc. CAD (Closed),NULL,F,NULL,CAD,NULL,NULL,NaN,0.0000,...,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,N
20590,x10,Steelworkers Pension Plan,NULL,F,NULL,CAD,NULL,NULL,57957192.0,0.0025,...,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,N
20591,x168x,Boilermakers Private Placements (Closed),NULL,F,NULL,CAD,NULL,NULL,NaN,0.0000,...,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,N


# 5. Changing relevant datatypes

In [8]:
import pandas as pd

def suggest_datatype(series):
    if pd.api.types.is_integer_dtype(series):
        return 'INTEGER'
    elif pd.api.types.is_float_dtype(series):
        return 'FLOAT'
    elif pd.api.types.is_datetime64_any_dtype(series):
        return 'DATETIME'
    elif pd.api.types.is_string_dtype(series):
        return 'VARCHAR(MAX)'
    else:
        return 'VARCHAR(MAX)'
        
columns_in_df = df_cleaned.columns

data_types = {col: suggest_datatype(df_cleaned[col]) for col in columns_in_df}
print(data_types)



{'ACCT_CD': 'VARCHAR(MAX)', 'ACCT_NAME': 'VARCHAR(MAX)', 'ACCT_SHT_NAME': 'VARCHAR(MAX)', 'ACCT_TYP_CD': 'VARCHAR(MAX)', 'PARENT_CHILD_FLAG': 'VARCHAR(MAX)', 'CRRNCY_CD': 'VARCHAR(MAX)', 'STATE_CD': 'VARCHAR(MAX)', 'CNTRY_CD': 'VARCHAR(MAX)', 'MKT_VAL': 'FLOAT', 'NET_ASSETS': 'FLOAT', 'TOT_ASSETS': 'FLOAT', 'FUND_SHRS_OUTST': 'VARCHAR(MAX)', 'TOT_COST': 'FLOAT', 'NET_CASH': 'VARCHAR(MAX)', 'TOT_INVSTMNTS': 'FLOAT', 'NET_FUNDS_AVAIL': 'VARCHAR(MAX)', 'LIABILITIES': 'VARCHAR(MAX)', 'AMRTZD_COST': 'VARCHAR(MAX)', 'AVG_COST': 'VARCHAR(MAX)', 'OTH_ASSET': 'VARCHAR(MAX)', 'DIV_RECEIVED': 'VARCHAR(MAX)', 'INT_RECEIVED': 'VARCHAR(MAX)', 'CNTRBS': 'VARCHAR(MAX)', 'PMNTS': 'VARCHAR(MAX)', 'TRANSFERS': 'VARCHAR(MAX)', 'CASH_BAL_SOD': 'VARCHAR(MAX)', 'RECVB_SEC_SOLD': 'VARCHAR(MAX)', 'PAYBL_SEC_PURCH': 'VARCHAR(MAX)', 'RECVB_FUND_SHRS_SOLD': 'VARCHAR(MAX)', 'PAYBL_FUND_SHRS_LIQD': 'VARCHAR(MAX)', 'DIV_RCVBLE': 'VARCHAR(MAX)', 'ACCRUED_INT_INCM': 'VARCHAR(MAX)', 'ACCRUED_EXP_INCM': 'VARCHAR(MAX)', 

# 6. Creating Index


In [10]:
import pyodbc
import pandas as pd

server = 'DELL\\SQLEXPRESS'
database = 'NEHADB'

connection_string = f"DRIVER={{ODBC Driver 17 for SQL Server}};SERVER={server};DATABASE={database};Trusted_Connection=yes;"
conn = pyodbc.connect(connection_string)

cursor = conn.cursor()

# Query to dynamically get the primary key or unique column(s) for the table dbo.APXENRICHED_FUND
sql_get_index_column = """
SELECT c.name AS column_name
FROM sys.index_columns ic
JOIN sys.columns c ON ic.column_id = c.column_id AND ic.object_id = c.object_id
JOIN sys.indexes i ON ic.object_id = i.object_id AND ic.index_id = i.index_id
WHERE i.is_primary_key = 1 AND i.object_id = OBJECT_ID('dbo.CS_FUND');
"""

cursor.execute(sql_get_index_column)
index_column = cursor.fetchone()

if index_column:
    index_column = index_column[0]  # Fetch the column name

    # Dynamically create an index if it does not exist
    sql_create_index = f"""
    IF NOT EXISTS (
        SELECT 1 
        FROM sys.indexes 
        WHERE object_id = OBJECT_ID('dbo.CS_FUND') 
        AND name = 'Index_CF{index_column}'
    )
    BEGIN
        CREATE INDEX Index_CF{index_column}
        ON dbo.CS_FUND({index_column});
    END
    """

    cursor.execute(sql_create_index)
    conn.commit()

    # Select the first 5 records
    sql_select_head = "SELECT TOP 5 * FROM dbo.CS_FUND"
    data = pd.read_sql(sql_select_head, conn)

    # Reset index and rename the default index column
    data = data.reset_index().rename(columns={'index': 'Index'})
    print(data.head())
else:
    print("No primary key or unique index found for the table dbo.CS_FUND.")

cursor.close()
conn.close()


   Index ACCT_CD                                          ACCT_NAME  \
0      0    006x  FCA Canada Inc. Elected Master Trust (Equity)(...   
1      1    007x  Canadian Pacific Airline Pilots' Association (...   
2      2    008x                Grosvenor International Canada Ltd.   
3      3    015x                             Johnson & Johnson Inc.   
4      4    020x  Province of Newfoundland and Labrador Pooled P...   

  ACCT_SHT_NAME ACCT_TYP_CD PARENT_CHILD_FLAG CRRNCY_CD STATE_CD CNTRY_CD  \
0          NULL           F              NULL       CAD     NULL     NULL   
1          NULL           F              NULL       CAD     NULL     NULL   
2          NULL           F              NULL       CAD     NULL     NULL   
3          NULL           F              NULL       CAD     NULL     NULL   
4          NULL           F              NULL       CAD     NULL     NULL   

  MKT_VAL  ...  TARGET_DURATION  FACTOR_MODEL_HIERARCHY_CD  \
0    None  ...             NULL                 

C:\Users\Neha\AppData\Local\Temp\ipykernel_22712\2066358798.py:46: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data = pd.read_sql(sql_select_head, conn)
